In [7]:
from IPython.display import HTML

HTML("""
<style>
    .custom-header {
        color: blue;
        font-size: 24px;
        font-weight: bold;
    }
    .custom-subheader {
        color:pink;
        font-size: 20px;
        font-weight: bold
        font-type: italian;
    }
</style>
<h1 class="custom-header">Project Title: Virtual Assistant Using Python</h1>
<h2 class="custom-subheader">Introduction</h2>
<p>Virtual assistance refers to the provision of administrative, technical, or creative support services to clients remotely..<br>
<h2>Services provided by our Virtual Assistance:-</h2><br>
 1) Greats you <br>
 2) Tells weather of cities <br>
 3) Gives  information from wikipedia<br> 
 4) Translator of different languages <br>
 5) Set reminder<br>
 6) Text to speech <br>
 7) Tells current time<br></p>
""")


# Installing Libraries

In [8]:
pip install speechrecognition

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install pyttsx3

In [10]:
pip install pyttsx3

In [12]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install pyaudio

In [23]:
pip install azure-storage-blob



  Obtaining dependency information for azure-storage-blob from https://files.pythonhosted.org/packages/15/19/2be26569e708cb618feecd7316ee0c5475273f7cdb4f9030a862870c74c8/azure_storage_blob-12.20.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/392.2 kB ? eta -:--:--
   ---------------------------- ----------- 276.5/392.2 kB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 392.2/392.2 kB 6.2 MB/s eta 0:00:00


In [6]:
!pip install azure-functions azure-storage-blob pandas

  Obtaining dependency information for azure-functions from https://files.pythonhosted.org/packages/cc/df/517aa6884bd7090d16d8f845b09cdc8b6e6057a95d06129966a92bc5c1f2/azure_functions-1.20.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/181.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/181.4 kB ? eta -:--:--
   -------------------------------------- - 174.1/181.4 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 181.4/181.4 kB 2.8 MB/s eta 0:00:00


In [2]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


# commands:-
   
    ## hi /hello
    ##weather in city
    ##read text hi how are you...
    ##food wikipedia 
    ##what time is it (current time only)
    ## translate, text (you want to convert),destination code (en,fr,es..)
    ##exit
    
# Language Codes    
    ##Google Translate uses ISO 639-1 language codes. Here are some examples:

  English: 'en' ,
  Spanish: 'es',
  French: 'fr' ,
  German: 'de',
  Chinese: 'zh-cn' ,
  Japanese: 'ja'
    
    
   

In [3]:
import speech_recognition as sr
import pyttsx3
import threading
import datetime
import time
import requests
from bs4 import BeautifulSoup
from googletrans import Translator  # Import Translator from googletrans

# Initialize the text-to-speech engine
def init_speech_engine():
    engine = pyttsx3.init()
    return engine

# Speak the given text
def speak(engine, text):
    engine.say(text)
    engine.runAndWait()

# Listen to the user's query
def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = r.listen(source)
        try:
            query = r.recognize_google(audio, language='en-in')
            print(f"User said: {query}\n")
            return query.lower()
        except sr.UnknownValueError:
            print("Sorry, I did not understand that. Please say that again.")
        except sr.RequestError as e:
            print(f"Could not request results; {e}")
    return None

# Fetch weather information
def fetch_weather(location):
    api_key = "14dfb89aa4d46a580298c85a4b2969b2"  # Replace with your OpenWeatherMap API key
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + location + "&units=metric"
    response = requests.get(complete_url)
    data = response.json()
    if response.status_code == 200 and "main" in data:
        main = data["main"]
        temperature = main["temp"]
        pressure = main["pressure"]
        humidity = main["humidity"]
        weather_desc = data["weather"][0]["description"]
        return f"Temperature: {temperature}°C\nPressure: {pressure} hPa\nHumidity: {humidity}%\nDescription: {weather_desc}"
    elif data.get("message"):
        return f"Error: {data['message']}"
    else:
        return "An unexpected error occurred while fetching the weather information."

# Fetch Wikipedia summary using BeautifulSoup
def fetch_wikipedia(query):
    try:
        search_url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"
        response = requests.get(search_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            paragraphs = soup.find_all('p')
            if paragraphs:
                first_paragraph = paragraphs[0].get_text()
                return first_paragraph, search_url
            else:
                return "No content found on Wikipedia for the given query.", search_url
        else:
            return f"Error fetching Wikipedia page: {response.status_code}", search_url
    except Exception as e:
        return f"An error occurred while fetching data from Wikipedia: {e}", search_url

# Translate text
def translate_text(text, dest_language):
    translator = Translator()
    translation = translator.translate(text, dest=dest_language)
    return translation.text

# Simple chatbot responses
def chatbot_response(query, engine, stop_event):
    if 'hello' in query or 'hi' in query:
        return "Hello! How can I assist you today?"
    elif 'what time is it' in query:
        now = datetime.datetime.now()
        return f"The current time is {now.strftime('%H:%M')}"
    elif 'weather in' in query:
        location = query.split('in')[-1].strip()
        return fetch_weather(location)
    elif 'wikipedia' in query:
        search_term = query.replace('wikipedia', '').strip()
        content, url = fetch_wikipedia(search_term)
        return f"{content}\nYou can read more here: {url}"
    elif 'read text' in query:
        text_to_read = query.replace('read text', '').strip()
        return text_to_read
    elif 'translate' in query:
        try:
            _, text_to_translate, dest_lang = query.split(',')
            translated_text = translate_text(text_to_translate.strip(), dest_lang.strip())
            return f"Original text: {text_to_translate.strip()}\nTranslated text: {translated_text}"
        except ValueError:
            return "Please provide the text and destination language code in the format: translate, <text>, <destination_language_code>"
    elif 'exit' in query or 'stop' in query:
        stop_event.set()
        speak(engine, "Goodbye!")
        return None
    else:
        return "I'm sorry, I can only assist with telling the time, fetching weather information, Wikipedia searches, reading text, and translating text at the moment."

# Function for text-based chat
def text_chat():
    while True:
        query = input("You: ").strip().lower()
        if query:
            response = chatbot_response(query, engine, stop_event)
            if response is None:
                break
            print(f"Assistant: {response}")
            speak(engine, response)
        else:
            print("Please enter a valid query.")

if __name__ == "__main__":
    engine = init_speech_engine()
    stop_event = threading.Event()

    # Choose interaction mode
    mode = input("Choose interaction mode: 'voice' or 'text': ").strip().lower()
    
    if mode == "voice":
        while True:
            query = listen()
            if query:
                response = chatbot_response(query, engine, stop_event)
                if response is None:
                    break
                speak(engine, response)
            else:
                speak(engine, "Please say that again.")
    elif mode == "text":
        text_chat()
    else:
        print("Invalid mode selected. Please restart the program and choose either 'voice' or 'text'.")

    print("Program exited cleanly.")

    

Choose interaction mode: 'voice' or 'text': text
You: hi
Assistant: Hello! How can I assist you today?
You: You: translate, Hello, fr
Assistant: Hello! How can I assist you today?
You: translate, Hello, fr
Assistant: Hello! How can I assist you today?
You: You: translate, bye, fr
Assistant: Original text: bye
Translated text: au revoir
You: translate, kese ho yaaar ,en
Assistant: Original text: kese ho yaaar
Translated text: How are you dear
You: what time is it
Assistant: The current time is 22:21
You: exit
Program exited cleanly.


# Setting Reminder

In [1]:
import datetime
import pandas as pd
import io
from azure.storage.blob import BlobServiceClient

# Azure Storage credentials and container name
connection_string = "DefaultEndpointsProtocol=https;AccountName=chatbotpractice313;AccountKey=Ib45vjzSUga1dCTFb3k36uN8QBRSgXCkAv0K7+OWCQoWmvfn+at0OizjiS7IK5AuhZR8Z/senDfN+ASt11jzNA==;EndpointSuffix=core.windows.net"
container_name = "virtualassistance313"

# Initialize Azure Blob service client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# Function to greet the user
def greet_user():
    current_time = datetime.datetime.now()
    if current_time.hour < 12:
        return "Good morning! How can I assist you today?"
    elif 12 <= current_time.hour < 18:
        return "Good afternoon! What can I do for you?"
    else:
        return "Good evening! How can I help you?"

# Function to authenticate the user using Azure AD (dummy implementation)
def authenticate_user(username, password):
    # Placeholder authentication logic (replace with actual Azure AD authentication)
    if username == "admin" and password == "password":
        return True
    else:
        return False

# Function to add a reminder
def add_reminder(reminder_text, reminder_time):
    reminder_data = {
        'reminder_text': reminder_text,
        'reminder_time': reminder_time,
        'created_at': datetime.datetime.now().isoformat()
    }
    csv_data = pd.DataFrame([reminder_data]).to_csv(index=False)
    blob_name = f"reminder_{reminder_data['created_at']}.csv"
    blob_client = container_client.get_blob_client(blob_name)
    
    try:
        blob_client.upload_blob(csv_data, overwrite=True)
        print(f"Reminder added: '{reminder_text}' at {reminder_time}")
    except Exception as e:
        print(f"Failed to add reminder: {e}")

# Function to list all reminders
def list_reminders():
    try:
        blob_list = container_client.list_blobs()
        reminders_list = []
        
        for blob in blob_list:
            if blob.name.startswith("reminder_"):
                blob_client = container_client.get_blob_client(blob.name)
                blob_data = blob_client.download_blob().readall()
                reminder_df = pd.read_csv(io.BytesIO(blob_data))
                reminders_list.append(reminder_df)
        
        if not reminders_list:
            return "No reminders found."
        else:
            return pd.concat(reminders_list).to_string(index=False)
    
    except Exception as e:
        return f"Failed to list reminders: {e}"

# Function to delete a reminder
def delete_reminder(reminder_created_at):
    try:
        blob_name = f"reminder_{reminder_created_at}.csv"
        blob_client = container_client.get_blob_client(blob_name)
        
        if blob_client.exists():
            blob_client.delete_blob()
            print(f"Reminder deleted: {reminder_created_at}")
        else:
            print("Reminder not found.")
    
    except Exception as e:
        print(f"Failed to delete reminder: {e}")

# Function to schedule a meeting
def schedule_meeting(meeting_subject, meeting_time):
    meeting_data = {
        'meeting_subject': meeting_subject,
        'meeting_time': meeting_time,
        'created_at': datetime.datetime.now().isoformat()
    }
    csv_data = pd.DataFrame([meeting_data]).to_csv(index=False)
    blob_name = f"meeting_{meeting_data['created_at']}.csv"
    blob_client = container_client.get_blob_client(blob_name)
    
    try:
        blob_client.upload_blob(csv_data, overwrite=True)
        print(f"Meeting scheduled: '{meeting_subject}' at {meeting_time}")
    except Exception as e:
        print(f"Failed to schedule meeting: {e}")

# Function to list all meetings
def list_meetings():
    try:
        blob_list = container_client.list_blobs()
        meetings_list = []
        
        for blob in blob_list:
            if blob.name.startswith("meeting_"):
                blob_client = container_client.get_blob_client(blob.name)
                blob_data = blob_client.download_blob().readall()
                meeting_df = pd.read_csv(io.BytesIO(blob_data))
                meetings_list.append(meeting_df)
        
        if not meetings_list:
            return "No meetings scheduled."
        else:
            return pd.concat(meetings_list).to_string(index=False)
    
    except Exception as e:
        return f"Failed to list meetings: {e}"

# Function to delete a meeting
def delete_meeting(meeting_created_at):
    try:
        blob_name = f"meeting_{meeting_created_at}.csv"
        blob_client = container_client.get_blob_client(blob_name)
        
        if blob_client.exists():
            blob_client.delete_blob()
            print(f"Meeting deleted: {meeting_created_at}")
        else:
            print("Meeting not found.")
    
    except Exception as e:
        print(f"Failed to delete meeting: {e}")

# Function to handle reminders menu
def handle_reminders():
    while True:
        action = input("What would you like to do with reminders? (add/list/delete/exit) ").strip().lower()

        if action == "add":
            reminder_text = input("What is the reminder for? ")
            reminder_time = input("When should I remind you? (format: YYYY-MM-DDTHH:MM:SS) ")
            add_reminder(reminder_text, reminder_time)

        elif action == "list":
            print(list_reminders())

        elif action == "delete":
            reminder_created_at = input("Enter the created_at time of the reminder to delete (format: YYYY-MM-DDTHH:MM:SS): ")
            delete_reminder(reminder_created_at)

        elif action == "exit":
            print("Exiting reminders menu...")
            break

        else:
            print("Invalid action. Please choose add, list, delete, or exit.")

# Function to handle meetings menu
def handle_meetings():
    while True:
        action = input("What would you like to do with meetings? (schedule/list/delete/exit) ").strip().lower()

        if action == "schedule":
            meeting_subject = input("What is the meeting about? ")
            meeting_time = input("When is the meeting? (format: YYYY-MM-DDTHH:MM:SS) ")
            schedule_meeting(meeting_subject, meeting_time)

        elif action == "list":
            print(list_meetings())

        elif action == "delete":
            meeting_created_at = input("Enter the created_at time of the meeting to delete (format: YYYY-MM-DDTHH:MM:SS): ")
            delete_meeting(meeting_created_at)

        elif action == "exit":
            print("Exiting meetings menu...")
            break

        else:
            print("Invalid action. Please choose schedule, list, delete, or exit.")

# Function to handle user interaction
def handle_interaction():
    print(greet_user())
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    if not authenticate_user(username, password):
        print("Authentication failed!")
        return

    while True:
        main_action = input("What would you like to do? (reminders/meetings/exit) ").strip().lower()

        if main_action == "reminders":
            handle_reminders()

        elif main_action == "meetings":
            handle_meetings()

        elif main_action == "exit":
            print("Exiting...")
            break

        else:
            print("Invalid action. Please choose reminders, meetings, or exit.")

# Main function to start the virtual assistant
def start_virtual_assistant():
    print("Hello! Welcome to the Virtual Assistant.")
    handle_interaction()

if __name__ == "__main__":
    start_virtual_assistant()


Hello! Welcome to the Virtual Assistant.
Good evening! How can I help you?
Enter your username: admin
Enter your password: password
What would you like to do? (reminders/meetings/exit) reminders
What would you like to do with reminders? (add/list/delete/exit) add
What is the reminder for? hnmm
When should I remind you? (format: YYYY-MM-DDTHH:MM:SS) 2024-07-09T04:06:09
Reminder added: 'hnmm' at 2024-07-09T04:06:09
What would you like to do with reminders? (add/list/delete/exit) exit
Exiting reminders menu...
What would you like to do? (reminders/meetings/exit) exit
Exiting...


In [2]:
from IPython.display import HTML

HTML("""
<style>
    .custom-header {
        color: blue;
        font-size: 24px;
        font-weight: bold;
    }
    .custom-subheader {
        color:  blue;
        font-size: 20px;
        font-weight: bold;
    }
</style>
<h1>Results and Discussion</h1>
<h3> This project demonstrates the creation of a virtual assistant that can handle tasks such as setting reminders, fetching weather information, and searching Wikipedia. The assistant can interact with users both through voice and text-based interfaces.
</h3>
""")



#  References
- [OpenWeatherMap API](https://openweathermap.org/api)
- [Wikipedia API](https://www.mediawiki.org/wiki/API:Main_page)
- [SpeechRecognition Library](https://pypi.org/project/SpeechRecognition/)
- [pyttsx3 Library](https://pypi.org/project)
- azure platform (container, storage account)